# File for previewing state, metro, county, and tract Census Codes
### Exported as .py file @ scripts/FetchCensusCodes.py to enable importing of functions and variables

In [75]:
# Native
import os
import re

# Third Party
import pprint
import censusdata
import pandas as pd

In [76]:
survey = "acs5"
year = 2018
state = "Missouri"
metro_area = "St. Louis"
MO_metro_counties = ["Franklin", "Jefferson", "Lincoln", "St. Charles", "St. Louis city", "St. Louis", "Warren"]
IL_metro_counties = ["Bond", "Calhoun", "Clinton", "Jersey", "Macoupin", "Madison", "Monroe", "St. Clair"]

In [77]:
try:
    census_api_key = os.environ["census_api_key"]
except:
    census_api_key = None

In [78]:
def fetch_metro_area_code(metro_area, survey=survey, year=year):
    geo = censusdata.censusgeo([('metropolitan statistical area/micropolitan statistical area', '*')])
    metro_areas = censusdata.geographies(geo, survey, year, key=census_api_key)
    for metro, code in metro_areas.items():
        if metro.startswith(metro_area):
            return re.search(r'\d+$', str(code)).group()
    return None

In [79]:
def fetch_state_code(state, survey=survey, year=year):
    geo = censusdata.censusgeo([('state', '*')])
    response = censusdata.geographies(geo, survey, year, key=census_api_key)
    state_values = str(response[state])
    state_code = re.search(r"\d+$", state_values)
    return state_code.group()

In [80]:
def fetch_county_code(county, state=state, survey=survey, year=year):
    geo = censusdata.censusgeo([('county', '*')])
    response = censusdata.geographies(geo, survey, year, key=census_api_key)
    
    try:
        _county_ = f"{county} County, {state}"
        county_values = str(response[_county_])
    except Exception as err:
        _county_ = f"{county}, {state}"
        county_values = str(response[_county_])
        
    county_code = re.search(r"\d+$", county_values)
    return county_code.group()

In [81]:
def fetch_tract_codes_by_county(state_code, county_code, survey=survey, year=year):
    geo = censusdata.censusgeo([('state', state_code), ('county', county_code), ('tract', '*')])
    tracts = censusdata.geographies(geo, survey, year, key=census_api_key)
    tract_codes = []
    for tract, code in tracts.items():
        code = re.search(r'\d+$', str(code)).group()
        tract_codes.append((tract, code))
    return tract_codes

## STL Metro and State Codes

In [82]:
metro_code = fetch_metro_area_code(metro_area)
MO_state_code = fetch_state_code(state)
IL_state_code = fetch_state_code(state="Illinois")
print(f"STL Metro Code: {metro_code}")
print(f"MO State Code: {MO_state_code}")
print(f"IL State Code: {IL_state_code}")

STL Metro Code: 41180
MO State Code: 29
IL State Code: 17


## All Missouri County Codes

In [83]:
geo = censusdata.censusgeo([('state', MO_state_code), ('county', '*')])
MO_counties = censusdata.geographies(geo, survey, year, key=census_api_key)
pprint.pp(MO_counties)

{'Saline County, Missouri': censusgeo((('state', '29'), ('county', '195'))),
 'Madison County, Missouri': censusgeo((('state', '29'), ('county', '123'))),
 'Wright County, Missouri': censusgeo((('state', '29'), ('county', '229'))),
 'Vernon County, Missouri': censusgeo((('state', '29'), ('county', '217'))),
 'Nodaway County, Missouri': censusgeo((('state', '29'), ('county', '147'))),
 'Chariton County, Missouri': censusgeo((('state', '29'), ('county', '041'))),
 'Grundy County, Missouri': censusgeo((('state', '29'), ('county', '079'))),
 'Morgan County, Missouri': censusgeo((('state', '29'), ('county', '141'))),
 'Wayne County, Missouri': censusgeo((('state', '29'), ('county', '223'))),
 'Bollinger County, Missouri': censusgeo((('state', '29'), ('county', '017'))),
 'Cape Girardeau County, Missouri': censusgeo((('state', '29'), ('county', '031'))),
 'Shelby County, Missouri': censusgeo((('state', '29'), ('county', '205'))),
 'Webster County, Missouri': censusgeo((('state', '29'), ('cou

## All Illinois County Codes

In [84]:
geo = censusdata.censusgeo([('state', IL_state_code), ('county', '*')])
IL_counties = censusdata.geographies(geo, survey, year, key=census_api_key)
pprint.pp(IL_counties)

{'Jersey County, Illinois': censusgeo((('state', '17'), ('county', '083'))),
 'Putnam County, Illinois': censusgeo((('state', '17'), ('county', '155'))),
 'De Witt County, Illinois': censusgeo((('state', '17'), ('county', '039'))),
 'Fayette County, Illinois': censusgeo((('state', '17'), ('county', '051'))),
 'Lee County, Illinois': censusgeo((('state', '17'), ('county', '103'))),
 'Logan County, Illinois': censusgeo((('state', '17'), ('county', '107'))),
 'Macoupin County, Illinois': censusgeo((('state', '17'), ('county', '117'))),
 'Richland County, Illinois': censusgeo((('state', '17'), ('county', '159'))),
 'Saline County, Illinois': censusgeo((('state', '17'), ('county', '165'))),
 'Washington County, Illinois': censusgeo((('state', '17'), ('county', '189'))),
 'Montgomery County, Illinois': censusgeo((('state', '17'), ('county', '135'))),
 'Whiteside County, Illinois': censusgeo((('state', '17'), ('county', '195'))),
 'Adams County, Illinois': censusgeo((('state', '17'), ('county

## STL Metro County Codes (Missouri and Illinois)

In [85]:
MO_metro_county_codes = [fetch_county_code(county) for county in MO_metro_counties]
IL_metro_county_codes = [fetch_county_code(county, state="Illinois") for county in IL_metro_counties]
pprint.pp(dict(zip(MO_metro_counties, MO_metro_county_codes)))
pprint.pp(dict(zip(IL_metro_counties, IL_metro_county_codes)))

{'Franklin': '071',
 'Jefferson': '099',
 'Lincoln': '113',
 'St. Charles': '183',
 'St. Louis city': '510',
 'St. Louis': '189',
 'Warren': '219'}
{'Bond': '005',
 'Calhoun': '013',
 'Clinton': '027',
 'Jersey': '083',
 'Macoupin': '117',
 'Madison': '119',
 'Monroe': '133',
 'St. Clair': '163'}


## All Tract Codes for STL Metro (Saint Louis)

In [86]:
MO_metro_tract_codes = [fetch_tract_codes_by_county(MO_state_code, county_code) for county_code in MO_metro_county_codes]
IL_metro_tract_codes = [fetch_tract_codes_by_county(IL_state_code, county_code) for county_code in IL_metro_county_codes]
pprint.pp(MO_metro_tract_codes)
pprint.pp(IL_metro_tract_codes)

[[('Census Tract 8001, Franklin County, Missouri', '800100'),
  ('Census Tract 8008, Franklin County, Missouri', '800800'),
  ('Census Tract 8002.01, Franklin County, Missouri', '800201'),
  ('Census Tract 8004.01, Franklin County, Missouri', '800401'),
  ('Census Tract 8007.01, Franklin County, Missouri', '800701'),
  ('Census Tract 8007.02, Franklin County, Missouri', '800702'),
  ('Census Tract 8003, Franklin County, Missouri', '800300'),
  ('Census Tract 8006.02, Franklin County, Missouri', '800602'),
  ('Census Tract 8010, Franklin County, Missouri', '801000'),
  ('Census Tract 8002.02, Franklin County, Missouri', '800202'),
  ('Census Tract 8005, Franklin County, Missouri', '800500'),
  ('Census Tract 8004.02, Franklin County, Missouri', '800402'),
  ('Census Tract 8006.01, Franklin County, Missouri', '800601'),
  ('Census Tract 8009.01, Franklin County, Missouri', '800901'),
  ('Census Tract 8009.02, Franklin County, Missouri', '800902'),
  ('Census Tract 8011.01, Franklin Count